In [3]:
import numpy as np
import random
import pandas as pd
import warnings
import math

In [33]:
def add(x1, x2):
    return np.add(x1, x2)

def subtract(x1, x2):
    return np.subtract(x1, x2)

def multiply(x1, x2):
    return np.multiply(x1, x2)

def divide(x1, x2):
    return np.divide(x1, x2)

def power(x1, x2):
    return np.power(x1, x2)

binary_functions = [add, subtract, multiply, divide, power]                                                                                                                                         

def log(x):
    return np.log(x)

def log10(x):
    return np.log10(x)

def arccos(x):
    return np.arccos(x)

def arcsin(x):
    return np.arcsin(x)

def sqrt(x):
    return np.sqrt(x)

unary_functions = [
    np.sin, np.cos, np.exp, np.abs, np.square,
    log, log10, sqrt,
    arcsin, arccos
]

constants = lambda: random.uniform(-10, 10)
constants_nozero = lambda: random.uniform(-10, 10) or constants_nozero()
constants_positive = lambda: random.uniform(0, 10)
constants_positive_nozero = lambda: random.uniform(0, 10) or constants_positive_nozero()
constants_interval = lambda: random.uniform(-1, 1)

#constants = lambda: random.uniform(-1000, 1000)
#constants_nozero = lambda: random.uniform(-1000, 1000) or constants_nozero()
#constants_positive = lambda: random.uniform(0, 1000)
#constants_positive_nozero = lambda: random.uniform(0, 1000) or constants_positive_nozero()


In [5]:
def calculate_correlation(x, y):
    data = {}
    for i in range(x.shape[0]):
        data[f"x{i+1}"] = x[i] 
    
    data["y"] = y
    matrix = pd.DataFrame(data)
    return matrix.corr()['y']

In [37]:
# Function to generate a random tree
def generate_random_tree(depth, x_dim, corr_vector, linearly_correlated_vars, used_linear_var=None, parent=None):

    # Calculate the non-linear variables
    non_linear_vars = [i for i in range(x_dim) if i not in linearly_correlated_vars]

    # Initialize the set of already used linear variables
    if used_linear_var is None:
        used_linear_var = set()

    # Base case: leaf
    if depth == 0:
        # If the parent is a binary function --> node is either a variable or a non-zero constant
        if parent in binary_functions:

            if non_linear_vars and random.random() > 0.4:
                index_non_linear_var = random.choice(non_linear_vars)
                return ("x", index_non_linear_var, None, None, parent)
            
            else: 
                const = constants_nozero()
                return ("const", const, None, None, parent)
               
        # If the parent is a unary function --> node is either a variable or a constant 
        elif parent in unary_functions:
            if parent == unary_functions[5] or parent == unary_functions[6]: #log and log10:

                if non_linear_vars and random.random() > 0.4:
                    index_non_linear_var = random.choice(non_linear_vars)
                    return ("x", index_non_linear_var, None, None, parent)
                
                else:
                    const = constants_positive()
                    return ("const", const, None, None, parent)
                
            elif parent == unary_functions[7]: #sqrt:

                if non_linear_vars and random.random() > 0.4:
                    index_non_linear_var = random.choice(non_linear_vars)
                    return ("x", index_non_linear_var, None, None, parent)
                
                else:
                    const = constants_positive_nozero()
                    return ("const", const, None, None, parent)
                
            elif parent == unary_functions[8] or parent == unary_functions[9]: #arcsin e arccos

                if non_linear_vars and random.random() > 0.4:
                    index_non_linear_var = random.choice(non_linear_vars)
                    return ("x", index_non_linear_var, None, None, parent)
                
                else:
                    const = constants_interval()
                    return ("const", const, None, None, parent)
            else:
                
                if non_linear_vars and random.random() > 0.4:
                    index_non_linear_var = random.choice(non_linear_vars)
                    return ("x", index_non_linear_var, None, None, parent)
                
                else:
                    const = constants_interval()
                    return ("const", const, None, None, parent)
            
        elif parent == "x":
            return None
        elif parent == "const":
            return None

    # Linear part
    if linearly_correlated_vars and not used_linear_var.intersection(linearly_correlated_vars) and parent not in unary_functions:
        linear_index = random.choice([v for v in linearly_correlated_vars if v not in used_linear_var])
        used_linear_var.add(linear_index)
        operator = binary_functions[0] if corr_vector[linear_index] >= 0 else binary_functions[1]
        left = ("x", linear_index, None, None, operator)
        right = generate_random_tree(depth - 1, x_dim, corr_vector, linearly_correlated_vars, used_linear_var, operator)
        return ("op", operator, left, right, parent)


# Binary node
    if random.random() > 0.5:
        operator = random.choice(binary_functions)

        # For np.power, constrain the exponent to manageable values
        if operator == binary_functions[4]: #power
            # Generate base and exponent
            base = generate_random_tree(depth - 1, x_dim, corr_vector, linearly_correlated_vars, used_linear_var, operator)
            exponent = ("const", random.uniform(0.5, 3), None, None, operator)
            return ("op", operator, base, exponent, parent)
        
        if operator == binary_functions[3]: #divide
            numerator = generate_random_tree(depth - 1, x_dim, corr_vector, linearly_correlated_vars, used_linear_var, operator)
            const = constants_nozero()
            denominator = ("const", const, None, None, parent)
            return ("op", operator, numerator, denominator, parent)

        # # For other operators
        left = generate_random_tree(depth - 1, x_dim, corr_vector, linearly_correlated_vars, used_linear_var, operator)
        right = generate_random_tree(depth - 1, x_dim, corr_vector, linearly_correlated_vars, used_linear_var, operator)
        return ("op", operator, left, right, parent)
    
    else:
        # Unary node
        operator = random.choice(unary_functions)
        child = generate_random_tree(depth - 1, x_dim, corr_vector, linearly_correlated_vars, used_linear_var, operator)

        return ("op", operator, child, None, parent)

In [7]:
def evaluate(tree, x):
    if tree[0] == "x":
        return x[tree[1]]
    elif tree[0] == "const":
        return tree[1]
    elif tree[0] == "op":
        func = tree[1]
        left = evaluate(tree[2], x) if tree[2] else None
        right = evaluate(tree[3], x) if tree[3] else None
        if left is not None and right is not None:
            return func(left, right)
        elif left is not None:
            return func(left)
        else:
            raise ValueError("Operator without valid arguments")
    else:
        raise ValueError("Invalid node")

def tree_to_string(tree):
    if tree[0] == "x":
        return f"x[{tree[1]}]"
    elif tree[0] == "const":
        return str(tree[1])
    elif tree[0] == "op":
        func_name = tree[1].__name__
        left = tree_to_string(tree[2]) if tree[2] else ""
        right = tree_to_string(tree[3]) if tree[3] else ""
        if tree[3] is None:  # unary node
            return f"{func_name}({left})"
        return f"({left} {func_name} {right})"
    else:
        return "Invalid node"

In [8]:
def fitness(tree, data, target):
    # MSE
    predictions = np.array([evaluate(tree, data)])
    return np.square(target-predictions).sum()/len(target)

In [9]:
def select_random_subtree(tree, linearly_correlated_vars, is_root=True):
    """
    Returns a random subtree from the given tree, avoiding selecting the entire tree
    as a subtree when called from the root.

    :param tree: The tree from which to extract the subtree
    :param linearly_correlated_vars: variables that have a strong linear correlation with the output that should be excluded in certain selections
    :param is_root: Flag to indicate if we are at the root of the tree
    :return: A random subtree    
    
    Remaninder - Tree structure:
    - tree[0]: Indicates the type of node ("op", "x", or "const").
    - tree[1]: If the node is "op", this indicates the operator type (e.g., addition, multiplication).
    - tree[2]: Left child of the node (can be None if the node has no left child).
    - tree[3]: Right child of the node (can be None, especially for unary nodes).
    """
    if tree is None:  # If the node is None, there are no subtrees to return
        return None

    # When we are at the root of the tree
    if is_root:
        # Binary node: try to descend to one of the children
        if tree[0] == "op" and tree[1] in binary_functions:
            # 50% chance to explore the left child
            if random.random() < 0.5 and tree[2] is not None and tree[2][1] not in linearly_correlated_vars:
                return select_random_subtree(tree[2], linearly_correlated_vars, is_root=False)  # Explore the left child
            
            # Otherwise, explore the right child if it exists
            elif tree[3] is not None:
                return select_random_subtree(tree[3], linearly_correlated_vars, is_root=False)  # Explore the right child

        # Unary node: explore the only child if it exists
        elif tree[0] == "op" and tree[1] in unary_functions and tree[2] is not None:
            return select_random_subtree(tree[2], linearly_correlated_vars, is_root=False)

        # If no valid children, return None to force another selection
        return None

    # When we are not at the root, there is a 30% chance to select the current node
    if random.random() < 0.3:
        return tree

    # If we don't select the current node, explore its children
    if tree[0] == "op" and tree[1] in binary_functions:
        # 50% chance to explore the left child
        if random.random() < 0.5 and tree[2] is not None and tree[2][1] not in linearly_correlated_vars:
            return select_random_subtree(tree[2], linearly_correlated_vars, is_root=False)
        
        # Otherwise, explore the right child if it exists
        elif tree[3] is not None:
            return select_random_subtree(tree[3], linearly_correlated_vars, is_root=False)

    # Unary node: explore the only child if it exists
    elif tree[0] == "op" and tree[1] in unary_functions and tree[2] is not None:
        return select_random_subtree(tree[2], linearly_correlated_vars, is_root=False)

    # If none of the conditions are met, return the current node
    return tree


In [10]:
def crossover(tree1, tree2, linearly_correlated_vars):
    """
    Perform a crossover operation between two trees, selecting random subtrees from each parent and swapping them.
    
    :param tree1 (tuple): The first parent tree from which a random subtree will be selected.
    :param tree2 (tuple): The second parent tree from which a random subtree will be selected.
    :param linearly_correlated_vars (list): A list of variables that have a strong linear correlation with the output, 
                                             used to guide the selection of subtrees.
    :return: A new tree generated by swapping subtrees from tree1 and tree2.
    """
    
    # Select a random subtree from each parent tree
    subtree1 = select_random_subtree(tree1, linearly_correlated_vars )
    subtree2 = select_random_subtree(tree2, linearly_correlated_vars )

    # If either subtree is None, it means we cannot perform the crossover
    if subtree1 is None or subtree2 is None:
        return tree1 if subtree1 is None else tree2  # Return one of the parents if no subtree was found

    # Swap the subtree of tree1 with that of tree2 and vice versa
    child1 = replace_subtree(tree1, subtree1, subtree2)
    child2 = replace_subtree(tree2, subtree2, subtree1)

    # Return the combined offspring (you can choose to return one of the modified parents)
    return child1 if random.random() > 0.5 else child2



def replace_subtree(tree, old_subtree, new_subtree):
    """
    Recursive function that replaces a subtree with another subtree
    
    :param tree (tuple): represents the main tree in which the subtree to be replaced will be searched. 
    :param old_subtree (tuple): the subtree to be searched and replaced in the main tree. 
    It must have the same structure and node type as the one to be replaced.
    :param new_subtree (tuple): the new subtree that will replace the occurrence of old_subtree 
    in the main tree.
    return: new_tree
    
    """
    
    # If the current tree is equal to the subtree we want to replace
    if tree == old_subtree:
        return new_subtree  # Return the new subtree (replacement)
    
    # If the current node is an operator node ("op"), explore the children
    if tree[0] == "op":
        # Check the children of the node, recursively replacing the subtree in the left and right children
        left = replace_subtree(tree[2], old_subtree, new_subtree) if tree[2] else None  # Replace in the left child
        right = replace_subtree(tree[3], old_subtree, new_subtree) if tree[3] else None  # Replace in the right child
        
        # Return the tree with modified children (if necessary)
        return (tree[0], tree[1], left, right, tree[4])  # Return the "op" node with the modified children

    return tree  # If the current node is not the one to replace, return it unchanged

In [11]:
# Initial population
def generate_population(size, depth, x_dim, corr_vector, linearly_correlated_vars):
    return [generate_random_tree(depth, x_dim, corr_vector, linearly_correlated_vars) for _ in range(size)]

# Evaluate the fitness of each individual in the population
def evaluate_population_fitness(population, x, y):
    fitness_scores = []
    invalid_count = 0 
    mse_na = 0
    for tree in population:
        try:
            score = fitness(tree, x, y)
            if math.isnan(score):
                fitness_scores.append((float('inf'), tree)) 
                mse_na += 1 # Penalize invalid trees with mse NaN
            else:
                    fitness_scores.append((score, tree))
        except Exception:
            fitness_scores.append((float('inf'), tree))  # Penalize invalid trees
            invalid_count += 1
    return sorted(fitness_scores, key=lambda x: x[0])  # Sort by fitness ascending

# elite selection
def select_top_individuals(sorted_population, elite_size):
    return [tree for _, tree in sorted_population[:elite_size]]

# Funzione principale dell'algoritmo di GP
def genetic_programming(x, y, x_dim, corr_vector, linearly_correlated_vars , POPULATION_SIZE, GENERATIONS, DEPTH):
    
    population = generate_population(POPULATION_SIZE, DEPTH, x_dim, corr_vector, linearly_correlated_vars)
    
    for generation in range(GENERATIONS):
        # Evaluation
        sorted_population = evaluate_population_fitness(population, x, y)
        
        # Selection
        elite_size = max(1, POPULATION_SIZE // 5)
        elite = select_top_individuals(sorted_population, elite_size)

        # Population reconstruction with new random trees
        new_population = elite[:]
        while len(new_population) < POPULATION_SIZE:
            parent1, parent2 = random.sample(elite, 2) if random.random() > 0.5 else elite[:2]
            # apply the crossover
            child = crossover(parent1, parent2, linearly_correlated_vars)
            new_population.append(child)
          
        population = new_population
        
    #print(f"Miglior albero: {tree_to_string(sorted_population[0][1])}")
    # Restituisce il miglior individuo finale
    return sorted_population[0][1]


In [12]:
def generate_function(x, y, DEPTH):
    
    warnings.filterwarnings("ignore")
    
    POPULATION_SIZE = 2_000
    GENERATIONS = 50
    x_dim = x.ndim
    corr_vector = calculate_correlation(x,y)

    threshold = 0.95  ## Threshold for considering strong correlation
    linearly_correlated_vars  = [i for i, corr in enumerate(corr_vector[:-1]) if abs(corr) >= threshold]

    return genetic_programming(x, y, x_dim, corr_vector, linearly_correlated_vars , POPULATION_SIZE, GENERATIONS, DEPTH)

In [14]:
def select_best_tree(num_experiments, DEPTH, x, y):
    best_tree = None
    best_fitness = float('inf')
    
    for _ in range(num_experiments):
        # Generate random tree
        tree = generate_function(x, y, DEPTH)
        
        # Calculate the fitness
        mse = fitness(tree, x, y)
        
        # Update the best
        if mse < best_fitness:
            best_fitness = mse
            best_tree = tree
            
    return best_tree, best_fitness

## TEST

## Problem 0

In [16]:
problem = np.load('problem_0.npz')
x = problem['x']
y = problem['y']

num_experiments = 4
DEPTH = 2
best_tree, best_fitness = select_best_tree(num_experiments, DEPTH, x, y)
print('Miglior albero finale: ', best_tree, '   ', best_fitness)

Miglior albero: (x[0] add sin(sin(sin(sin(sin(sin((x[1] divide 5.523348392778454))))))))
Miglior albero: (x[0] add sin(sin(sin(sin(sin(sin(sin(sin(sin((x[1] divide 5.439059926717535)))))))))))
Miglior albero: (x[0] add (sin(x[1]) divide 5.07429872773508))
Miglior albero: (x[0] add (sin(x[1]) divide 5.279603984762815))
Miglior albero finale:  ('op', <function add at 0x00000208DFF75EE0>, ('x', 0, None, None, <function add at 0x00000208DFF75EE0>), ('op', <function divide at 0x00000208E1301DC0>, ('op', <ufunc 'sin'>, ('x', 1, None, None, <ufunc 'sin'>), None, <function add at 0x00000208DFF75EE0>), ('const', 5.07429872773508, None, None, <function divide at 0x00000208E1301DC0>), <function add at 0x00000208DFF75EE0>), None)     2.2887881131780577e-06


In [17]:
print(tree_to_string(best_tree))
print(f"MSE (train): {100*np.square(y-evaluate(best_tree,x)).sum()/len(y):g}")

(x[0] add (sin(x[1]) divide 5.07429872773508))
MSE (train): 0.000228879


In [20]:
def f_problem_0(x):
    return np.add(x[0], np.divide(np.sin(x[1]), 5.07429872773508))

print(f"MSE (train): {100*np.square(y-f_problem_0(x)).sum()/len(y):g}")

MSE (train): 0.000228879


## Problem 1

In [21]:
problem = np.load('problem_1.npz')
x = problem['x']
y = problem['y']

num_experiments = 5
DEPTH = 2
best_tree, best_fitness = select_best_tree(num_experiments, DEPTH, x, y)
print('Miglior albero finale: ', best_tree, '   ', best_fitness)

Miglior albero: (x[0] add (3.2048373254236022 power (-4.289515392996326 subtract 1.9191224129558744)))
Miglior albero: (x[0] add square((-0.06936638636469183 divide 2.4664072463578783)))
Miglior albero: (x[0] add ((9.123482043626172 subtract 9.06489629010407) power 2.632163151688034))
Miglior albero: (x[0] add (((5.749671429445183 add -5.67397010796906) divide 3.7789748952531337) power 1.7788581495214206))
Miglior albero: (x[0] add exp(-7.353284879164015))
Miglior albero finale:  ('op', <function add at 0x00000208DFF75EE0>, ('x', 0, None, None, <function add at 0x00000208DFF75EE0>), ('op', <ufunc 'square'>, ('op', <function divide at 0x00000208E1301DC0>, ('const', -0.06936638636469183, None, None, <function divide at 0x00000208E1301DC0>), ('const', 2.4664072463578783, None, None, <function power at 0x00000208E1301E50>), <function add at 0x00000208DFF75EE0>), None, <function add at 0x00000208DFF75EE0>), None)     0.003280862643283991


In [56]:
print(tree_to_string(best_tree))
print(f"MSE (train): {100*np.square(y-evaluate(best_tree,x)).sum()/len(y):g}")

(x[0] add ((0.10854911946931267 divide 3.7265487206860755) multiply (0.10854911946931267 divide 3.7265487206860755)))
MSE (train): 0.328086


In [22]:
def f_problem_1(x):
    return np.add(x[0], np.multiply(np.divide(0.10854911946931267, 3.7265487206860755),
                                  np.divide(0.10854911946931267, 3.7265487206860755)))

print(f"MSE (train): {100*np.square(y-f_problem_1(x)).sum()/len(y):g}")

MSE (train): 0.328086


## Problem 2

In [23]:
problem = np.load('problem_2.npz')
x = problem['x']
y = problem['y']

num_experiments = 5
DEPTH = 2
best_tree, best_fitness = select_best_tree(num_experiments, DEPTH, x, y)
print('Miglior albero finale: ', best_tree, '   ', best_fitness)

Miglior albero: (exp(x[0]) power 2.9989266179928737)
Miglior albero: exp((x[0] add 9.70435665670923))
Miglior albero: square(((x[0] add 9.787716499141705) power 2.8008242718725995))
Miglior albero: (8.81525747355741 multiply exp((x[0] add 8.81525747355741)))
Miglior albero: (exp((x[0] subtract -9.237079355907301)) multiply x[0])
Miglior albero finale:  ('op', <ufunc 'square'>, ('op', <function power at 0x00000208E1301E50>, ('op', <function add at 0x00000208DFF75EE0>, ('x', 0, None, None, <ufunc 'exp'>), ('const', 9.787716499141705, None, None, <function add at 0x00000208DFF75EE0>), <ufunc 'exp'>), ('const', 2.8008242718725995, None, None, <function power at 0x00000208E1301E50>), <ufunc 'square'>), None, None)     25693231654223.438


In [24]:
print(tree_to_string(best_tree))
print(f"MSE (train): {100*np.square(y-evaluate(best_tree,x)).sum()/len(y):g}")

square(((x[0] add 9.787716499141705) power 2.8008242718725995))
MSE (train): 2.56932e+15


In [26]:
problem = np.load('problem_2.npz')
x = problem['x']
y = problem['y']

num_experiments = 4
DEPTH = 3
best_tree, best_fitness = select_best_tree(num_experiments, DEPTH, x, y)
print('Miglior albero finale: ', best_tree, '   ', best_fitness)

Miglior albero: (((9.503037074443881 subtract -7.944359702123512) multiply ((9.503037074443881 subtract -7.944359702123512) multiply ((9.503037074443881 subtract -7.944359702123512) multiply exp(x[0])))) add square(((9.503037074443881 subtract -7.944359702123512) multiply exp(x[0]))))
Miglior albero: (square((8.09037463120157 multiply 8.09037463120157)) multiply ((8.09037463120157 subtract -8.594501415099812) multiply ((8.09037463120157 subtract -8.594501415099812) multiply x[0])))
Miglior albero: (((x[0] add 8.41029184409922) power 2.171525067136721) power 2.743365526145127)
Miglior albero: ((exp(x[0]) multiply (9.499449293900184 power 1.78177746724181)) power 1.78177746724181)
Miglior albero finale:  ('op', <function multiply at 0x00000208E1301D30>, ('op', <ufunc 'square'>, ('op', <function multiply at 0x00000208E1301D30>, ('const', 8.09037463120157, None, None, <function add at 0x00000208DFF75EE0>), ('const', 8.09037463120157, None, None, <function add at 0x00000208DFF75EE0>), <ufun

In [27]:
print(tree_to_string(best_tree))
print(f"MSE (train): {100*np.square(y-evaluate(best_tree,x)).sum()/len(y):g}")

(square((8.09037463120157 multiply 8.09037463120157)) multiply ((8.09037463120157 subtract -8.594501415099812) multiply ((8.09037463120157 subtract -8.594501415099812) multiply x[0])))
MSE (train): 1.90536e+15


In [37]:
def f_problem_2(x):
    return np.multiply(
    np.square(np.multiply(8.09037463120157, 8.09037463120157)),
    np.multiply(
        np.subtract(8.09037463120157, -8.594501415099812),
        np.multiply(
            np.subtract(8.09037463120157, -8.594501415099812),
            x[0]
        )
    )
)

print(f"MSE (train): {100*np.square(y-f_problem_2(x)).sum()/len(y):g}")

MSE (train): 1.90536e+15


## Problema 3

In [38]:
problem = np.load('problem_3.npz')
x = problem['x']
y = problem['y']

num_experiments = 5
DEPTH = 2
best_tree, best_fitness = select_best_tree(num_experiments, DEPTH, x, y)
print('Miglior albero finale: ', best_tree, '   ', best_fitness)

Miglior albero: ((x[1] multiply -8.890642119385188) add (-4.249875913580546 multiply (-4.249875913580546 add x[1])))
Miglior albero: (((((x[1] multiply -6.876894251787691) subtract -6.876894251787691) subtract -6.876894251787691) subtract -6.876894251787691) add (-6.876894251787691 multiply x[1]))
Miglior albero: ((((((((-9.999675389282633 multiply x[1]) subtract -9.999675389282633) subtract x[1]) subtract x[1]) subtract x[1]) subtract x[1]) subtract x[1]) subtract -9.999675389282633)
Miglior albero: ((-9.986229791122907 multiply x[1]) subtract ((x[1] add (x[1] add -6.823779785678996)) add ((x[1] add (x[1] add -6.823779785678996)) add (x[1] add -6.823779785678996))))
Miglior albero: ((1.145270977369592 subtract x[1]) multiply (x[1] multiply x[1]))
Miglior albero finale:  ('op', <function multiply at 0x00000208E1301D30>, ('op', <function subtract at 0x00000208DFF8A3A0>, ('const', 1.145270977369592, None, None, <function power at 0x00000208E1301E50>), ('x', 1, None, None, <function subtr

In [39]:
print(tree_to_string(best_tree))
print(f"MSE (train): {100*np.square(y-evaluate(best_tree,x)).sum()/len(y):g}")

((1.145270977369592 subtract x[1]) multiply (x[1] multiply x[1]))
MSE (train): 50611.4


### prova 2

In [40]:
problem = np.load('problem_3.npz')
x = problem['x']
y = problem['y']

num_experiments = 10
DEPTH = 2
best_tree, best_fitness = select_best_tree(num_experiments, DEPTH, x, y)
print('Miglior albero finale: ', best_tree, '   ', best_fitness)

Miglior albero: ((x[0] multiply x[0]) subtract (x[1] multiply (x[1] multiply (x[1] subtract 0.661028631104097))))
Miglior albero: (square((0.15028421051650653 subtract x[1])) multiply (((x[1] add -3.10335659101427) divide (x[1] add -3.10335659101427)) subtract x[1]))
Miglior albero: (9.855757791912954 subtract (9.855757791912954 multiply x[1]))
Miglior albero: (square((-5.538820314730466 add x[1])) subtract 4.634860071397707)
Miglior albero: ((x[1] multiply -9.56226970849389) add ((((((x[1] multiply -2.626733114394291) subtract -9.56226970849389) subtract x[1]) subtract x[1]) subtract x[1]) subtract -9.56226970849389))
Miglior albero: (((2.684249119983356 subtract x[1]) subtract x[1]) multiply 6.794030738775003)
Miglior albero: (9.051781066926 subtract (x[1] multiply (x[1] multiply x[1])))
Miglior albero: ((x[1] multiply -9.615601651101057) subtract (5.790067507430589 multiply (x[1] subtract 3.8269761857243605)))
Miglior albero: ((3.394325375896706 multiply 3.394325375896706) subtract 

In [41]:
print(tree_to_string(best_tree))
print(f"MSE (train): {100*np.square(y-evaluate(best_tree,x)).sum()/len(y):g}")

((x[0] multiply x[0]) subtract (x[1] multiply (x[1] multiply (x[1] subtract 0.661028631104097))))
MSE (train): 22132


### prova 3

In [42]:
problem = np.load('problem_3.npz')
x = problem['x']
y = problem['y']

num_experiments = 20
DEPTH = 2
best_tree, best_fitness = select_best_tree(num_experiments, DEPTH, x, y)
print('Miglior albero finale: ', best_tree, '   ', best_fitness)

Miglior albero: ((x[1] multiply -9.005220623715367) add square(x[0]))
Miglior albero: (square((3.5680737511272618 subtract x[1])) add (x[1] multiply -5.58431602437051))
Miglior albero: (absolute(1.8544315513756282) multiply ((x[1] subtract 1.8544315513756282) multiply -7.657877888827049))
Miglior albero: (square(x[0]) add (x[1] multiply -9.17148711216081))
Miglior albero: (square(x[0]) add (square(x[0]) subtract (x[1] multiply square(x[1]))))
Miglior albero: ((x[1] multiply x[1]) subtract ((-6.586782539647789 subtract (-6.586782539647789 multiply x[1])) subtract (-6.586782539647789 multiply x[1])))
Miglior albero: ((-9.541225532177817 multiply x[1]) add (((-3.147400615542237 add (x[1] subtract 2.8302915212444866)) multiply x[1]) subtract -9.541225532177817))
Miglior albero: (square(x[1]) multiply ((x[1] add x[1]) multiply -0.3724232366290554))
Miglior albero: (square(x[0]) add (((-9.746079233741137 multiply 1.7576928697068632) multiply x[1]) subtract (-9.746079233741137 multiply 1.7576

In [43]:
print(tree_to_string(best_tree))
print(f"MSE (train): {100*np.square(y-evaluate(best_tree,x)).sum()/len(y):g}")

(square(x[0]) add (square(x[0]) subtract (x[1] multiply square(x[1]))))
MSE (train): 11668.9


### prova 4

In [46]:
problem = np.load('problem_3.npz')
x = problem['x']
y = problem['y']

num_experiments = 20
DEPTH = 2
best_tree, best_fitness = select_best_tree(num_experiments, DEPTH, x, y)
print('Miglior albero finale: ', best_tree, '   ', best_fitness)

Miglior albero finale:  ('op', <function subtract at 0x00000208DFF8A3A0>, ('op', <function add at 0x00000208DFF75EE0>, ('op', <function multiply at 0x00000208E1301D30>, ('x', 0, None, None, <function multiply at 0x00000208E1301D30>), ('x', 0, None, None, <function multiply at 0x00000208E1301D30>), <function add at 0x00000208DFF75EE0>), ('op', <function add at 0x00000208DFF75EE0>, ('op', <function multiply at 0x00000208E1301D30>, ('x', 0, None, None, <function multiply at 0x00000208E1301D30>), ('x', 0, None, None, <function multiply at 0x00000208E1301D30>), <function add at 0x00000208DFF75EE0>), ('const', 5.884993886295414, None, None, <function add at 0x00000208DFF75EE0>), <function subtract at 0x00000208DFF8A3A0>), <function subtract at 0x00000208DFF8A3A0>), ('op', <function multiply at 0x00000208E1301D30>, ('x', 1, None, None, <function multiply at 0x00000208E1301D30>), ('op', <function multiply at 0x00000208E1301D30>, ('x', 1, None, None, <function multiply at 0x00000208E1301D30>), 

In [47]:
print(tree_to_string(best_tree))
print(f"MSE (train): {100*np.square(y-evaluate(best_tree,x)).sum()/len(y):g}")

(((x[0] multiply x[0]) add ((x[0] multiply x[0]) add 5.884993886295414)) subtract (x[1] multiply (x[1] multiply x[1])))
MSE (train): 10291


In [58]:
def f_problem_3(x):
    return np.subtract(
    np.add(
        np.multiply(x[0], x[0]),
        np.add(np.multiply(x[0], x[0]), 5.884993886295414)
    ),
    np.multiply(x[1], np.multiply(x[1], x[1]))
)

print(f"MSE (train): {100*np.square(y-f_problem_3(x)).sum()/len(y):g}")

MSE (train): 10291


## Problema 4

In [60]:
problem = np.load('problem_4.npz')
x = problem['x']
y = problem['y']

num_experiments = 5
DEPTH = 2
best_tree, best_fitness = select_best_tree(num_experiments, DEPTH, x, y)
print('Miglior albero finale: ', best_tree, '   ', best_fitness)

Miglior albero: ((((4.124920724109721 add cos(x[1])) add cos(x[1])) add (((4.124920724109721 add cos(x[1])) add cos(x[1])) add cos(x[1]))) multiply cos(x[1]))
Miglior albero: exp(cos(x[1]))
Miglior albero: exp(cos(x[1]))
Miglior albero: (((2.8090181841947235 add (cos(x[1]) add 2.023027231836082)) add (2.8090181841947235 add (cos(x[1]) add 2.023027231836082))) multiply cos((2.8090181841947235 add (cos((2.8090181841947235 add (cos((2.8090181841947235 add (cos(x[1]) add 2.023027231836082))) add 2.023027231836082))) add 2.023027231836082))))
Miglior albero: (((cos(x[1]) subtract -5.028761511486659) subtract -5.028761511486659) multiply cos((cos(x[1]) subtract -5.028761511486659)))
Miglior albero finale:  ('op', <function multiply at 0x00000218AB8540D0>, ('op', <function add at 0x00000218AB83DF70>, ('op', <function add at 0x00000218AB83DF70>, ('const', 2.8090181841947235, None, None, <function add at 0x00000218AB83DF70>), ('op', <function add at 0x00000218AB83DF70>, ('op', <ufunc 'cos'>, ('

In [61]:
print(tree_to_string(best_tree))
print(f"MSE (train): {100*np.square(y-evaluate(best_tree,x)).sum()/len(y):g}")

(((2.8090181841947235 add (cos(x[1]) add 2.023027231836082)) add (2.8090181841947235 add (cos(x[1]) add 2.023027231836082))) multiply cos((2.8090181841947235 add (cos((2.8090181841947235 add (cos((2.8090181841947235 add (cos(x[1]) add 2.023027231836082))) add 2.023027231836082))) add 2.023027231836082))))
MSE (train): 33.4439


### prova 2

In [59]:
problem = np.load('problem_4.npz')
x = problem['x']
y = problem['y']

num_experiments = 5
DEPTH = 2
best_tree, best_fitness = select_best_tree(num_experiments, DEPTH, x, y)
print('Miglior albero finale: ', best_tree, '   ', best_fitness)

Miglior albero finale:  ('op', <function subtract at 0x00000208DFF8A3A0>, ('op', <function add at 0x00000208DFF75EE0>, ('op', <function add at 0x00000208DFF75EE0>, ('op', <function add at 0x00000208DFF75EE0>, ('op', <function add at 0x00000208DFF75EE0>, ('op', <function add at 0x00000208DFF75EE0>, ('op', <function add at 0x00000208DFF75EE0>, ('op', <function add at 0x00000208DFF75EE0>, ('const', 4.32315452587539, None, None, <function add at 0x00000208DFF75EE0>), ('op', <ufunc 'cos'>, ('x', 1, None, None, <ufunc 'cos'>), None, <ufunc 'exp'>), <function subtract at 0x00000208DFF8A3A0>), ('op', <ufunc 'cos'>, ('x', 1, None, None, <ufunc 'cos'>), None, <ufunc 'exp'>), <function subtract at 0x00000208DFF8A3A0>), ('op', <ufunc 'cos'>, ('x', 1, None, None, <ufunc 'cos'>), None, <ufunc 'exp'>), <function subtract at 0x00000208DFF8A3A0>), ('op', <ufunc 'cos'>, ('x', 1, None, None, <ufunc 'cos'>), None, <ufunc 'exp'>), <function subtract at 0x00000208DFF8A3A0>), ('op', <ufunc 'cos'>, ('x', 1, N

In [60]:
print(tree_to_string(best_tree))
print(f"MSE (train): {100*np.square(y-evaluate(best_tree,x)).sum()/len(y):g}")

((((((((4.32315452587539 add cos(x[1])) add cos(x[1])) add cos(x[1])) add cos(x[1])) add cos(x[1])) add cos(x[1])) add cos(x[1])) subtract absolute(cos(x[1])))
MSE (train): 36.4739


### prova 3

In [18]:
problem = np.load('problem_4.npz')
x = problem['x']
y = problem['y']

num_experiments = 10
DEPTH = 3
best_tree, best_fitness = select_best_tree(num_experiments, DEPTH, x, y)
print('Miglior albero finale: ', tree_to_string(best_tree))
print(f"MSE (train): {100*np.square(y-evaluate(best_tree,x)).sum()/len(y):g}")

Miglior albero finale:  ((absolute((6.117730529641008 add 0.5150538807928458)) multiply cos(x[1])) add ((((6.117730529641008 add 0.5150538807928458) add 0.5150538807928458) add ((((6.117730529641008 add 0.5150538807928458) add 0.5150538807928458) add (absolute((6.117730529641008 add 0.5150538807928458)) multiply cos(x[1]))) power 0.5150538807928458)) power 0.5150538807928458))
MSE (train): 7.51795


In [17]:
problem = np.load('problem_4.npz')
x = problem['x']
y = problem['y']

num_experiments = 10
DEPTH = 3
best_tree, best_fitness = select_best_tree(num_experiments, DEPTH, x, y)
print('Miglior albero finale: ', tree_to_string(best_tree))
print(f"MSE (train): {100*np.square(y-evaluate(best_tree,x)).sum()/len(y):g}")

Miglior albero finale:  ((((((((log(9.599800492150578) add cos(x[1])) add cos(x[1])) add cos(x[1])) add cos(x[1])) add cos(x[1])) add cos(x[1])) add cos(x[1])) add cos((x[1] subtract x[1])))
MSE (train): 6.87446


## Problem 5

In [62]:
problem = np.load('problem_5.npz')
x = problem['x']
y = problem['y']

num_experiments = 5
DEPTH = 2
best_tree, best_fitness = select_best_tree(num_experiments, DEPTH, x, y)
print('Miglior albero finale: ', best_tree, '   ', best_fitness)

Miglior albero: log((x[1] divide x[1]))
Miglior albero: square((x[0] subtract x[0]))
Miglior albero: absolute((x[0] subtract x[0]))
Miglior albero: ((x[0] subtract x[0]) power 0.5257047306245456)
Miglior albero: log10((x[1] divide x[1]))
Miglior albero finale:  ('op', <function log at 0x00000218A8F92EE0>, ('op', <function divide at 0x00000218AB854160>, ('x', 1, None, None, <function divide at 0x00000218AB854160>), ('x', 1, None, None, <function divide at 0x00000218AB854160>), <function log at 0x00000218A8F92EE0>), None, None)     5.572810232617333e-18


In [63]:
print(tree_to_string(best_tree))
print(f"MSE (train): {100*np.square(y-evaluate(best_tree,x)).sum()/len(y):g}")

log((x[1] divide x[1]))
MSE (train): 5.57281e-16


### prova 2

In [73]:
problem = np.load('problem_5.npz')
x = problem['x']
y = problem['y']

num_experiments = 5
DEPTH = 2
best_tree, best_fitness = select_best_tree(num_experiments, DEPTH, x, y)
print('Miglior albero finale: ', best_tree, '   ', best_fitness)

Miglior albero finale:  ('op', <function power at 0x00000208E1301E50>, ('op', <function subtract at 0x00000208DFF8A3A0>, ('x', 1, None, None, <function subtract at 0x00000208DFF8A3A0>), ('x', 1, None, None, <function subtract at 0x00000208DFF8A3A0>), <function power at 0x00000208E1301E50>), ('const', 1.203572808700951, None, None, <function power at 0x00000208E1301E50>), None)     5.572810232617333e-18


In [74]:
print(tree_to_string(best_tree))
print(f"MSE (train): {100*np.square(y-evaluate(best_tree,x)).sum()/len(y):g}")

((x[1] subtract x[1]) power 1.203572808700951)
MSE (train): 5.57281e-16


In [75]:
def f_problem_5(x):
    return np.power(
    np.subtract(x[1], x[1]),
    1.203572808700951
)

print(f"MSE (train): {100*np.square(y-f_problem_5(x)).sum()/len(y):g}")

MSE (train): 5.57281e-16


## Problem 6

In [64]:
problem = np.load('problem_6.npz')
x = problem['x']
y = problem['y']

num_experiments = 5
DEPTH = 2
best_tree, best_fitness = select_best_tree(num_experiments, DEPTH, x, y)
print('Miglior albero finale: ', best_tree, '   ', best_fitness)

Miglior albero: ((x[1] add x[1]) subtract x[0])
Miglior albero: (((x[1] subtract x[0]) subtract x[0]) add (x[0] add x[1]))
Miglior albero: (((x[1] add 0.8490975631076996) add (((x[1] add 0.8490975631076996) add (x[0] divide -4.470089586744139)) divide -4.470089586744139)) subtract (x[0] subtract x[1]))
Miglior albero: ((x[1] subtract x[0]) add ((x[1] subtract ((x[1] subtract x[0]) divide 7.038301194783468)) subtract ((x[1] subtract x[0]) divide 7.038301194783468)))
Miglior albero: ((x[1] multiply 1.6740943296502273) add (cos(4.867449659423141) subtract x[0]))
Miglior albero finale:  ('op', <function add at 0x00000218AB83DF70>, ('op', <function subtract at 0x00000218AB854040>, ('x', 1, None, None, <function subtract at 0x00000218AB854040>), ('x', 0, None, None, <function subtract at 0x00000218AB854040>), <function add at 0x00000218AB83DF70>), ('op', <function subtract at 0x00000218AB854040>, ('op', <function subtract at 0x00000218AB854040>, ('x', 1, None, None, <function subtract at 0x0

In [65]:
print(tree_to_string(best_tree))
print(f"MSE (train): {100*np.square(y-evaluate(best_tree,x)).sum()/len(y):g}")

((x[1] subtract x[0]) add ((x[1] subtract ((x[1] subtract x[0]) divide 7.038301194783468)) subtract ((x[1] subtract x[0]) divide 7.038301194783468)))
MSE (train): 0.779218


In [14]:
problem = np.load('problem_6.npz')
x = problem['x']
y = problem['y']

In [15]:
def f_problem_6(x):
    return np.add(np.subtract(x[1], x[0]), 
                np.subtract(np.subtract(x[1], np.divide(np.subtract(x[1], x[0]), 7.038301194783468)),
                            np.divide(np.subtract(x[1], x[0]), 7.038301194783468)))

print(f"MSE (train): {100*np.square(y-f_problem_6(x)).sum()/len(y):g}")

MSE (train): 0.779218


## Problem 7

In [66]:
problem = np.load('problem_7.npz')
x = problem['x']
y = problem['y']

num_experiments = 5
DEPTH = 2
best_tree, best_fitness = select_best_tree(num_experiments, DEPTH, x, y)
print('Miglior albero finale: ', best_tree, '   ', best_fitness)

Miglior albero: absolute((((x[1] multiply x[0]) multiply (x[1] multiply x[0])) multiply ((x[1] multiply x[0]) subtract -5.460645453304853)))
Miglior albero: square(((x[0] add x[1]) add (x[0] add x[1])))
Miglior albero: absolute(((x[0] multiply (x[0] multiply x[1])) multiply -7.839984542987541))
Miglior albero: (exp((x[0] multiply x[1])) multiply square(-2.1735315692220887))
Miglior albero: (square(x[1]) power (x[1] multiply x[0]))
Miglior albero finale:  ('op', <function multiply at 0x00000218AB8540D0>, ('op', <ufunc 'exp'>, ('op', <function multiply at 0x00000218AB8540D0>, ('x', 0, None, None, <function multiply at 0x00000218AB8540D0>), ('x', 1, None, None, <function multiply at 0x00000218AB8540D0>), <ufunc 'exp'>), None, <function multiply at 0x00000218AB8540D0>), ('op', <ufunc 'square'>, ('const', -2.1735315692220887, None, None, <function multiply at 0x00000218AB8540D0>), None, <function multiply at 0x00000218AB8540D0>), None)     330.69271304858205


In [67]:
print(tree_to_string(best_tree))
print(f"MSE (train): {100*np.square(y-evaluate(best_tree,x)).sum()/len(y):g}")

(exp((x[0] multiply x[1])) multiply square(-2.1735315692220887))
MSE (train): 33069.3


### prova 2

In [81]:
problem = np.load('problem_7.npz')
x = problem['x']
y = problem['y']

num_experiments = 10
DEPTH = 2
best_tree, best_fitness = select_best_tree(num_experiments, DEPTH, x, y)
print('Miglior albero finale: ', best_tree, '   ', best_fitness)

Miglior albero finale:  ('op', <function multiply at 0x00000208E1301D30>, ('const', 4.616523103938919, None, None, <function power at 0x00000208E1301E50>), ('op', <ufunc 'exp'>, ('op', <function multiply at 0x00000208E1301D30>, ('x', 1, None, None, <function multiply at 0x00000208E1301D30>), ('x', 0, None, None, <function multiply at 0x00000208E1301D30>), <ufunc 'absolute'>), None, <function multiply at 0x00000208E1301D30>), None)     330.1103345808995


In [82]:
print(tree_to_string(best_tree))
print(f"MSE (train): {100*np.square(y-evaluate(best_tree,x)).sum()/len(y):g}")

(4.616523103938919 multiply exp((x[1] multiply x[0])))
MSE (train): 33011


### prova 3

In [83]:
problem = np.load('problem_7.npz')
x = problem['x']
y = problem['y']

num_experiments = 10
DEPTH = 3
best_tree, best_fitness = select_best_tree(num_experiments, DEPTH, x, y)
print('Miglior albero finale: ', best_tree, '   ', best_fitness)

Miglior albero finale:  ('op', <function add at 0x00000208DFF75EE0>, ('op', <function subtract at 0x00000208DFF8A3A0>, ('op', <ufunc 'square'>, ('op', <function multiply at 0x00000208E1301D30>, ('op', <ufunc 'sin'>, ('x', 1, None, None, <function multiply at 0x00000208E1301D30>), None, <function multiply at 0x00000208E1301D30>), ('op', <function multiply at 0x00000208E1301D30>, ('op', <ufunc 'sin'>, ('x', 1, None, None, <function multiply at 0x00000208E1301D30>), None, <function multiply at 0x00000208E1301D30>), ('op', <function multiply at 0x00000208E1301D30>, ('op', <ufunc 'sin'>, ('x', 1, None, None, <function multiply at 0x00000208E1301D30>), None, <function multiply at 0x00000208E1301D30>), ('op', <function multiply at 0x00000208E1301D30>, ('op', <ufunc 'sin'>, ('x', 1, None, None, <function multiply at 0x00000208E1301D30>), None, <function multiply at 0x00000208E1301D30>), ('op', <function multiply at 0x00000208E1301D30>, ('op', <ufunc 'sin'>, ('x', 1, None, None, <function multi

In [84]:
print(tree_to_string(best_tree))
print(f"MSE (train): {100*np.square(y-evaluate(best_tree,x)).sum()/len(y):g}")

((square((sin(x[1]) multiply (sin(x[1]) multiply (sin(x[1]) multiply (sin(x[1]) multiply (sin(x[1]) multiply (sin(x[1]) multiply (sin(x[1]) multiply (sin(x[1]) multiply (sin(x[1]) multiply (sin(x[1]) multiply (sin(x[1]) multiply (sin(x[1]) multiply (sin(x[1]) multiply (sin(x[1]) multiply (sin(x[1]) multiply (sin(x[0]) multiply square((sin(x[1]) multiply (sin(x[0]) multiply (sin(x[1]) multiply (sin(x[1]) multiply (sin(x[0]) multiply (sin(x[0]) multiply absolute((sin(x[0]) multiply (sin(x[0]) multiply absolute((sin(x[0]) multiply (sin(x[1]) multiply absolute(absolute((x[0] multiply x[1]))))))))))))))))))))))))))))))))) subtract -6.654030157337885) add ((exp((x[0] multiply x[1])) subtract -6.654030157337885) multiply (x[0] multiply x[1])))
MSE (train): 31349.4


## Problem 8

In [27]:
problem = np.load('problem_8.npz')
x = problem['x']
y = problem['y']

num_experiments = 1
DEPTH = 3

best_tree, best_fitness = select_best_tree(num_experiments, DEPTH, x, y)
print('Miglior albero finale: ', tree_to_string(best_tree), '   ', best_fitness)
print(f"MSE (train): {100*np.square(y-evaluate(best_tree,x)).sum()/len(y):g}")

Miglior albero finale:  (23.82495597313178 subtract (square(-23.7940974885513) add (x[1] multiply x[1])))     22705839.260495044
MSE (train): 2.27058e+09


In [28]:
problem = np.load('problem_8.npz')
x = problem['x']
y = problem['y']

num_experiments = 5
DEPTH = 2

best_tree, best_fitness = select_best_tree(num_experiments, DEPTH, x, y)
print('Miglior albero finale: ', tree_to_string(best_tree), '   ', best_fitness)
print(f"MSE (train): {100*np.square(y-evaluate(best_tree,x)).sum()/len(y):g}")

Miglior albero finale:  (((((77.31121524249622 add 1.4018494655475018) add 1.9418212806343498) add 1.4018494655475018) add 1.9418212806343498) subtract (99.70869502552355 power 1.4018494655475018))     22705914.36559638
MSE (train): 2.27059e+09


In [30]:
problem = np.load('problem_8.npz')
x = problem['x']
y = problem['y']

num_experiments = 1
DEPTH = 5

best_tree, best_fitness = select_best_tree(num_experiments, DEPTH, x, y)
print('Miglior albero finale: ', tree_to_string(best_tree), '   ', best_fitness)
print(f"MSE (train): {100*np.square(y-evaluate(best_tree,x)).sum()/len(y):g}")

Miglior albero finale:  (square((square(((-17.36956496566218 subtract 91.95246462574642) divide -42.880831916293836)) subtract (sin(square((square(((-17.36956496566218 subtract 91.95246462574642) divide -42.880831916293836)) subtract ((square(((-17.36956496566218 subtract 91.95246462574642) divide -42.880831916293836)) subtract (sin(((-17.36956496566218 subtract 31.33195707647286) divide -42.880831916293836)) subtract 91.95246462574642)) subtract 91.95246462574642)))) subtract 91.95246462574642))) divide -17.36956496566218)     22705914.36560858
MSE (train): 2.27059e+09


In [31]:
problem = np.load('problem_8.npz')
x = problem['x']
y = problem['y']

num_experiments = 1
DEPTH = 6

best_tree, best_fitness = select_best_tree(num_experiments, DEPTH, x, y)
print('Miglior albero finale: ', tree_to_string(best_tree), '   ', best_fitness)
print(f"MSE (train): {100*np.square(y-evaluate(best_tree,x)).sum()/len(y):g}")

Miglior albero finale:  (square(square(2.307971005290181)) subtract (log(exp(absolute(absolute(((x[1] subtract x[1]) add sqrt(72.56293957235948)))))) power 2.97338769306789))     22705947.734172314
MSE (train): 2.27059e+09


In [32]:
problem = np.load('problem_8.npz')
x = problem['x']
y = problem['y']

num_experiments = 1
DEPTH = 7

best_tree, best_fitness = select_best_tree(num_experiments, DEPTH, x, y)
print('Miglior albero finale: ', tree_to_string(best_tree), '   ', best_fitness)
print(f"MSE (train): {100*np.square(y-evaluate(best_tree,x)).sum()/len(y):g}")

Miglior albero finale:  ((((-84.45000735277137 add (-84.45000735277137 add -46.21596215547494)) add -46.21596215547494) add (((-84.45000735277137 add (-84.45000735277137 add -46.21596215547494)) add -46.21596215547494) add -46.21596215547494)) subtract ((sin(sin(sin(sin(0.4731332244475872)))) power 2.9610127102077795) multiply ((-84.45000735277137 add (-84.45000735277137 add -46.21596215547494)) add -46.21596215547494)))     22705914.369668227
MSE (train): 2.27059e+09


In [36]:
# aumento costanti
problem = np.load('problem_8.npz')
x = problem['x']
y = problem['y']

num_experiments = 5
DEPTH = 3

best_tree, best_fitness = select_best_tree(num_experiments, DEPTH, x, y)
print('Miglior albero finale: ', tree_to_string(best_tree), '   ', best_fitness)
print(f"MSE (train): {100*np.square(y-evaluate(best_tree,x)).sum()/len(y):g}")

Miglior albero finale:  ((((123.53358234281495 subtract 718.2788914418215) divide 123.53358234281495) subtract (123.53358234281495 add 421.37832983296175)) subtract ((123.53358234281495 subtract 421.37832983296175) divide (x[1] multiply ((x[1] multiply -745.3360084664045) subtract 718.2788914418215))))     22704431.043448012
MSE (train): 2.27044e+09
